# Попробуем реализовать stcking  
## Создадим классы-обертки для xgboost, lightgbm, catboost -- таким образом обеспечим единообразие обработки. В качестве мета-алгоритма примем xgboost.  

# В итоге, STACKING НЕ ДАЛ УЛУЧШЕНИЯ результата!!!! Возможно, не подобрал гипер-параметры, особено lightgbm ((   
# НО, скорее всего, мало данных всего 9000 записей. 

In [27]:
!pip install lightgbm

You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.



In [28]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

import sklearn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import os
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor

import ast
import json

from sklearn.preprocessing import StandardScaler

import gensim
from gensim import corpora
from pprint import pprint
from gensim.models import LdaModel, LdaMulticore, Phrases

import gensim.downloader as api
from gensim.utils import simple_preprocess, lemmatize
import re
import logging


from itertools import chain
from collections import defaultdict
from operator import itemgetter

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA 

In [29]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

Python       : 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
Numpy        : 1.19.1


In [30]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [31]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Setup

In [33]:
VERSION    = 35
DIR_TRAIN  = './input/04-09-2020/' # подключил к ноутбуку свой внешний датасет
DIR_TEST   = './input/sf-dst-car-price/'
VAL_SIZE   = 0.1 #0.33   # 33%
N_FOLDS    = 5

CURR_YEAR = 2020
PRICE_IN_TIME = 0.89 #0.9 # За время соревнования (7- мес) цены на подержанные авто выросли
ROUND_C = int(5000)
# CATBOOST
ITERATIONS = 6000#4000
LR         = 0.1 #0.05# 0.05#0.1#0.05#0.1
OD_PVAL = 0.002 #0.002 
L2_REG = 4.5 #4


In [35]:
X_sub = pd.read_csv('C:/Users/YBNY3VWM/SF/SkillFactory/MODELS-CARS/v2/X_sub_all.csv')
X = pd.read_csv('C:/Users/YBNY3VWM/SF/SkillFactory/MODELS-CARS/v2/X_all.csv')
y = pd.read_csv('C:/Users/YBNY3VWM/SF/SkillFactory/MODELS-CARS/v2/y_all.csv')
cat_features = [] 
with open('C:/Users/YBNY3VWM/SF/SkillFactory/MODELS-CARS/v3-подрезаны фичи!/cat_features.npy', 'rb') as f:
    cat_features = list(np.load(f))

y.drop(['Unnamed: 0'], axis=1, inplace=True)
X.drop(['Unnamed: 0'], axis=1, inplace=True)
X_sub.drop(['Unnamed: 0'], axis=1, inplace=True)
y=y.values.ravel()

In [36]:
from sklearn.metrics import mean_squared_error


In [37]:
def xgb_mape(preds, dtrain):
   labels = dtrain.get_label()
   return('mape', np.mean(np.abs((labels - preds) / (labels + 1))))

In [39]:
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb

from sklearn.metrics import mean_squared_error

In [40]:
def xgb_mape(preds, dtrain):
   labels = dtrain.get_label()
   return('mape', np.mean(np.abs((labels - preds) / (labels + 1))))

In [41]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [42]:
def get_oof(clf, ntrain, ntest, kf, train, labels, test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((5, ntest))

    print('--->',kf.n_splits, )
    N_FOLDS=5
    splits = list(KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED).split(X, y))
    for i, (train_index, test_index) in tqdm(enumerate(splits), total=N_FOLDS,):
        print('i --->', i)
        x_tr = train.iloc[train_index]
        y_tr = labels.iloc[train_index]
        x_te = train.iloc[test_index]
        y_te = labels.iloc[test_index]

        clf.train(x_tr, y_tr, x_te, y_te)
        oof_train[test_index] = clf.predict(x_te)
        prd=clf.predict(test)
        oof_test_skf[i, :] = prd
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [44]:
class XgbWrapper(object):
    def __init__(self, seed=RANDOM_SEED, params=None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nrounds', 400)

    def train(self, xtra, ytra, xte, yte):
        
        dtrain = xgb.DMatrix(xtra, label=ytra)
        dvalid = xgb.DMatrix(xte, label=yte)
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds,
            watchlist, feval=xgb_mape, early_stopping_rounds=10 )

    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))

In [45]:
class LgbWrapper(object):
    def __init__(self, seed=RANDOM_SEED, cat_features='auto', params=None):
        self.param = params
        self.param['seed'] = seed
        self.param['categorical_feature'] = cat_features
        self.nrounds = params.pop('nrounds', 400)
        self.cat_features = cat_features

    def train(self, xtra, ytra, xte, yte):

        ytra = ytra.values.ravel()
        yte = yte.values.ravel()
        dtrain = lgb.Dataset(xtra, label=ytra)
        self.gbdt = lgb.train(self.param, dtrain, self.nrounds,  feval=xgb_mape)
        
    def predict(self, x):
        return self.gbdt.predict(x)

In [46]:
class CtbWrapper(object):
    def __init__(self, seed=RANDOM_SEED, cat_features=None, params=None):
        self.seed = seed
        self.nrounds = 300
        self.cat_features = cat_features

    def train(self, xtra, ytra, xte, yte):
        self.gbdt = ctb.CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          od_type = 'IncToDec',
                          od_pval = OD_PVAL,
                        #   #one_hot_max_size=10,
                        #   #random_strength=3,
                        #   l2_leaf_reg =L2_REG
                          )

        xtra = pd.DataFrame(xtra)
        ytra = pd.DataFrame(ytra)
        xte = pd.DataFrame(xte)
        yte = pd.DataFrame(yte)

        self.gbdt.fit(X=xtra, y=ytra, cat_features=self.cat_features,
                      eval_set=(xte, yte),
                      verbose_eval=100,
                      use_best_model=True)

    def predict(self, x):
        return self.gbdt.predict(x)

In [47]:
def model_1(train, labels, test, cat_features_in):

    ntrain = train.shape[0]
    ntest = test.shape[0]

    kf = KFold(ntrain, n_splits=5,
               shuffle=True, random_state=RANDOM_SEED)


    cg = CtbWrapper(seed=RANDOM_SEED, cat_features=cat_features_in, params=ctb_params)
    xg = XgbWrapper(seed=RANDOM_SEED, params=xgb_params)
    lg = LgbWrapper(seed=RANDOM_SEED, cat_features=cat_features_in, params=lgb_params)
    
    Xd = train.drop(['name', 'vehicleConfiguration'], axis = 1) 
    X_sub_d = test.drop(['name', 'vehicleConfiguration'], axis = 1) 
    
    Xd = pd.get_dummies(Xd, drop_first=True)
    X_sub_d = pd.get_dummies(X_sub_d, drop_first=True)
    cols = list(set(Xd.columns) & set(X_sub_d.columns))
    Xd = Xd[cols]
    X_sub_d  = X_sub_d [cols]


    cg_oof_train, cg_oof_test = get_oof(cg, ntrain, ntest, kf, train, labels, test)
    lg_oof_train, lg_oof_test = get_oof(lg, ntrain, ntest, kf, Xd, labels, X_sub_d)    
    xg_oof_train, xg_oof_test = get_oof(xg, ntrain, ntest, kf, Xd, labels, X_sub_d)
  

    def mean_absolute_percentage_error(y_true, y_pred): 
        return np.mean(np.abs((y_true - y_pred) / y_true))
        
    print("CG-CV: {}".format(mean_absolute_percentage_error(labels, cg_oof_train.ravel())))
    print("XG-CV: {}".format(mean_absolute_percentage_error(labels, xg_oof_train.ravel())))
    print("LG-CV: {}".format(mean_absolute_percentage_error(labels, lg_oof_train.ravel())))


    x_train = np.concatenate((cg_oof_train.reshape(-1,1), xg_oof_train.reshape(-1,1), lg_oof_train.reshape(-1,1)),axis=1)
    print('------>', np.shape(x_train))

    x_test = np.concatenate((cg_oof_test.reshape(-1,1), xg_oof_test.reshape(-1,1), lg_oof_test.reshape(-1,1)), axis=1)
    print('=============>', np.shape(x_test))

    np.save(arr=x_train, file='x_concat_train.npy')
    np.save(arr=x_test, file='x_concat_test.npy')
    np.save(arr=labels, file='y_labels.npy')

    return x_train, labels, x_test

In [48]:
def model_2():
    train = np.load('x_concat_train.npy')
    labels = np.load('y_labels.npy')
    test = np.load('x_concat_test.npy')

    dtrain = xgb.DMatrix(train, label=labels)
    dtest = xgb.DMatrix(test)

    xgb_params = {}
    xgb_params["objective"] = "reg:squarederror"
    xgb_params["eta"] = 0.01
    xgb_params["subsample"] = 0.9
    xgb_params["silent"] = 1
    xgb_params["max_depth"] = 5
    xgb_params['eval_metric'] = 'rmse'
    xgb_params['min_child_weight'] = 10
    xgb_params['seed'] = RANDOM_SEED

    res = xgb.cv(xgb_params, dtrain, num_boost_round=500, nfold=5, seed=RANDOM_SEED, stratified=False,
                 early_stopping_rounds=25, verbose_eval=10, show_stdv=True, feval=xgb_mape)
    

    best_nrounds = res.shape[0] - 1
    cv_mean = res.iloc[-1, 0]
    cv_std = res.iloc[-1, 1]

    print('')
    print(f'Ensemble-CV: {cv_mean}+/-{cv_std}')
    print('1----')
    bst = xgb.train(xgb_params, dtrain, best_nrounds)
    print('2----')
    
    preds = bst.predict(dtest)
    print('3----')
    return preds

In [49]:
X_sub = pd.read_csv('C:/Users/YBNY3VWM/SF/SkillFactory/MODELS-CARS/v3-подрезаны фичи!/X_sub_all.csv')
X = pd.read_csv('C:/Users/YBNY3VWM/SF/SkillFactory/MODELS-CARS/v3-подрезаны фичи!/X_all.csv')
y = pd.read_csv('C:/Users/YBNY3VWM/SF/SkillFactory/MODELS-CARS/v3-подрезаны фичи!/y_all.csv')
cat_features = [] 
with open('C:/Users/YBNY3VWM/SF/SkillFactory/MODELS-CARS/v3-подрезаны фичи!/cat_features.npy', 'rb') as f:
    cat_features = list(np.load(f))



y.drop(['Unnamed: 0'], axis=1, inplace=True)
X.drop(['Unnamed: 0'], axis=1, inplace=True)
X_sub.drop(['Unnamed: 0'], axis=1, inplace=True)
X['Комплектация_1'].fillna('-NaN-', inplace=True)
X_sub['Комплектация_1'].fillna('-NaN-', inplace=True)
y=y.values.ravel()
y = pd.Series(y).astype('float')
cat_features = [X.columns.get_loc(c) for c in cat_features if c in X]
from random import randrange
import re
X = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', 'x_'+str(randrange(1900))+'_'+str(randrange(100)) , x))
columns = X.columns
X_sub.columns = columns

In [50]:
lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'regression_l2',
    'metric': 'mape',
    'num_leaves': 96,
    'max_depth': 12,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.95,
    'bagging_freq': 5,
    'learning_rate': 0.1,
    'lambda_l1': 2,
    'lambda_l2': 42,
    'n_estimators' : 100,
    'max_depth' : -1, 
    #'early_stopping_round':20
 }

xgb_params = {
    'objective': 'reg:squarederror',
    'disable_default_eval_metric' : 1,
    #'eval_metric': 'mape',
    'learning_rate' : 0.1,
    'base_score': 0.5,
    'booster': 'gbtree',
    'colsample_bylevel': 1,
    'colsample_bynode': 1,
    'colsample_bytree': 0.7,
    'gamma': 0,
    'gpu_id': -1,
    'importance_type': 'gain',
    'interaction_constraints': '',
    'max_delta_step': 0,
    'max_depth': 9,
    'min_child_weight': 1,
    'missing': np.nan,
    'monotone_constraints': '()',
    'n_estimators': 60,
    'n_jobs': -1,
    'num_parallel_tree': 1,
    'random_state': RANDOM_SEED,
    'reg_alpha': 10,
    'reg_lambda': 3,
    'scale_pos_weight': 1,
    'subsample': 0.7,
    'tree_method': 'exact',
    'validate_parameters': 1,
    'verbosity': None,
    'alpha': 10
    }

ctb_params = {
    'iterations' : 4000,
    'learning_rate' : 0.05,
    'eval_metric' : 'MAPE',
    'random_seed' : RANDOM_SEED,
    'od_type' : 'IncToDec',
    'od_pval' : OD_PVAL,
    'l2_leaf_reg' : L2_REG
    }

In [51]:
x_train, labels, x_test = model_1(X, y, X_sub, cat_features)


---> 9809


i ---> 0
0:	learn: 1.1927966	test: 1.1361373	best: 1.1361373 (0)	total: 125ms	remaining: 12m 30s
100:	learn: 0.1605712	test: 0.1618782	best: 0.1618782 (100)	total: 11.5s	remaining: 11m 13s
200:	learn: 0.1400566	test: 0.1473202	best: 0.1473202 (200)	total: 22.2s	remaining: 10m 40s
300:	learn: 0.1298397	test: 0.1420379	best: 0.1418769 (298)	total: 33.9s	remaining: 10m 41s
400:	learn: 0.1226315	test: 0.1394884	best: 0.1394369 (396)	total: 45s	remaining: 10m 28s
500:	learn: 0.1177089	test: 0.1376663	best: 0.1376024 (494)	total: 56.5s	remaining: 10m 20s
600:	learn: 0.1128698	test: 0.1360119	best: 0.1360119 (600)	total: 1m 7s	remaining: 10m 8s
700:	learn: 0.1089805	test: 0.1352213	best: 0.1351757 (694)	total: 1m 19s	remaining: 9m 57s
800:	learn: 0.1056581	test: 0.1345567	best: 0.1344599 (793)	total: 1m 30s	remaining: 9m 46s
900:	learn: 0.1026367	test: 0.1341862	best: 0.1341697 (899)	total: 1m 42s	remaining: 9m 38s
1000:	learn: 0.0991465	test: 0.1337912	best: 0.1337757 (993)	total: 1m 53s	rem

i ---> 0
[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002409 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2300
[LightGBM] [Info] Number of data points in the train set: 7847, number of used features: 138
[LightGBM] [Info] Start training from score 1191612.081050
[LightGBM] [Warning]No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

i ---> 0
[23:12:01] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { importance_type, missing, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-mape:0.89608	eval-mape:0.89486
Multiple eval metrics have been passed: 'eval-mape' will be used for early stopping.

Will train until eval-mape hasn't improved in 10 rounds.
[1]	train-mape:0.80278	eval-mape:0.80143
[2]	train-mape:0.71966	eval-mape:0.71844
[3]	train-mape:0.64480	eval-mape:0.64397
[4]	train-mape:0.57885	eval-mape:0.57809
[5]	train-mape:0.51941	eval-mape:0.51875
[6]	train-mape:0.46640	eval-mape:0.46544
[7]	train-mape:0.41923	eval-mape:0.41852
[8]	train-mape:0.37682	eval-mape:0.37599
[9]	train-mape:0.33914	eval-mape:0.33954
[10]	train-mape:0.3061

In [52]:
preds = model_2()
preds

[23:12:10] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:12:10] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:12:10] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings bu

array([1466544.4 , 2253873.5 , 1118813.4 , ...,  484047.28, 1040933.44,
       1018714.25], dtype=float32)

In [53]:
preds

array([1466544.4 , 2253873.5 , 1118813.4 , ...,  484047.28, 1040933.44,
       1018714.25], dtype=float32)

In [54]:
pd.Series(preds).describe()

count    3.837000e+03
mean     1.479724e+06
std      1.170087e+06
min      1.305345e+05
25%      7.385658e+05
50%      1.114612e+06
75%      1.789001e+06
max      8.722406e+06
dtype: float64

In [55]:
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')
sample_submission['price'] = preds
sample_submission['price'] = sample_submission['price'].astype('float')
sample_submission.to_csv(f'fin_stack_submission_blend_v{VERSION}.csv', index=False)
sample_submission.head(10)

,id,price
0,0,1466544.375
1,1,2253873.500
2,2,1118813.375
3,3,2311837.250
4,4,4568785.500
5,5,1915298.250
6,6,1013205.250
7,7,622664.875
8,8,1314327.750
9,9,1319853.125
